In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pandas as pd
import random

import DataLoader
from Trainer import Trainer, initialize_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

In [3]:
set_seed(0)

In [4]:
# device - cpu or gpu?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Fine-tuning model**

In [7]:
feature_extract = False # fine-tuning
resnet18_model_fine_tuning, input_size = initialize_model('resnet', num_classes=400, feature_extract=feature_extract, use_pretrained=True)
resnet18_model_fine_tuning = resnet18_model_fine_tuning.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
# fine-tuning we will be updating all parameters. However, if we are
# doing feature extract method, we will only update the parameters
# that we have just initialized, i.e. the parameters with requires_grad
# is True.
params_to_update = resnet18_model_fine_tuning.parameters()
if feature_extract:
  params_to_update = []
  for name,param in resnet18_model_fine_tuning.named_parameters():
    if param.requires_grad == True:
      params_to_update.append(param)

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.98), eps=1e-09)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

resnet18_model_fine_tuning_trainer = Trainer(resnet18_model_fine_tuning, device, input_size, criterion, optimizer_ft, exp_lr_scheduler, batch_size=64, num_epochs=50, num_workers=5, dataset_dir='./drive/MyDrive/VoxCeleb1')

In [ ]:
resnet18_model_fine_tuning_trainer.train_model('train')

Epoch 0/49
----------


**Freezed model**

In [4]:
feature_extract = True # freezing
resnet18_model_freezed, input_size = initialize_model('resnet', num_classes=400, feature_extract=feature_extract, use_pretrained=True)
resnet18_model_freezed = resnet18_model_freezed.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
# fine-tuning we will be updating all parameters. However, if we are
# doing feature extract method, we will only update the parameters
# that we have just initialized, i.e. the parameters with requires_grad
# is True.
params_to_update = resnet18_model_freezed.parameters()
print("Params to learn:")
if feature_extract:
  params_to_update = []
  for name,param in resnet18_model_freezed.named_parameters():
    if param.requires_grad == True:
      params_to_update.append(param)
      print("\t",name)
else:
  for name,param in resnet18_model_freezed.named_parameters():
    if param.requires_grad == True:
      print("\t",name)
    
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.98), eps=1e-09)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

resnet18_freezed_trainer = Trainer(resnet18_model_freezed, device, input_size, criterion, optimizer_ft, exp_lr_scheduler, batch_size=32, num_epochs=1, num_workers=2, dataset_dir='./drive/MyDrive/VoxCeleb1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
resnet18_freezed_trainer.train_model('train')

Epoch 0/0
----------
train Loss: 14.6014 Acc: 0.0072
val Loss: 9.9899 Acc: 0.0027

Training complete in 2m 41s
Best val Acc: 0.002693


In [ ]:
resnet18_freezed_trainer.evaluate_model()